<a href="https://colab.research.google.com/github/AusafMo/CustomDocument-ChatBot/blob/main/LLM_Knowledgebase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPDF2

In [ ]:
!pip install spacy

In [ ]:
!python3 -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
import PyPDF2
import pandas as pd
import spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
path = '/content/drive/MyDrive/Sukh Dev - Prime Ayurvedic Plant Drugs-Springer-Ane Books (2023).pdf'

with open(path, 'rb') as pdf_file:

  pdf_reader = PyPDF2.PdfReader(pdf_file)

  num_pages = len(pdf_reader.pages)

  paragraphs = []
  print(num_pages)

  for i in range(num_pages):

    page = pdf_reader.pages[i]

    text = page.extract_text()

    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]

    paragraph = ''
    for sentence in sentences :
      if len(paragraph) + len(sentence) > 500:
        paragraphs.append(paragraph)
        paragraph = sentence
      else :
        paragraph += ' ' + sentence

    paragraphs.append(paragraph)

  df = pd.DataFrame({'content': paragraphs})



850


In [ ]:
df

,content
0,Prime \nAyurvedic \nPlant DrugsSukh Dev\nSeco...
1,Prime Ayurvedic Plant Drug s
2,Sukh Dev \nPrime Ayurvedic \nPlant Drug s \nS...
3,ISBN 978 -3-031-22074-6 ISBN 978-3-031-22075...
4,© Author 2016 \nThis work is subject to copyri...
...,...
4085,652\nTerminalia arjuna 661\nTerminalia bellir...
4086,
4087,"825\nS. Dev, Prime Ayurvedic Plant Drugs , htt..."
4088,


In [ ]:
# !pip install openai
!pip install tiktoken

In [ ]:
import numpy as np
import openai
import tiktoken
import time
from google.colab import userdata
openai.api_key = userdata.get('openai_key')

In [ ]:
COMPLETIONS_MODEL = 'text-davinci-003'
EMBEDDING_MODEL   = 'text-embedding-ada-002'

In [ ]:
def get_embedding(text :str, model: str = EMBEDDING_MODEL) -> list[float]:
  result = openai.Embedding.create( model = model, input = text)
  time.sleep(5)
  return result['data'][0]['embedding']


  def compute_doc_embeddings(df: pd.DataFrame) -> dict[tuple[str,str], list[float]]:
    return { idx : get_embedding(r.content) for idx, r in df.iterrows()}

In [ ]:
def load_embeddings(fname:str) -> dict[tuple[str,str], list[float]]:
  df = pd.read_csv(fname, header = 0)
  max_dim = max([int(c) for c in df.columns if c != 'title' and c!= 'heading'])
  return {
      (r.title, r.heading): [r[str(i)] for i in range(max_dim + 1)] for _, r in df.iterrows()
  }

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd
from sklearn.preprocessing import normalize

sentence_tf_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
def get_hf_embeddings(text: str, model) -> np.ndarray:
    sentence_embeddings = model.encode(text)
    sentence_embeddings = sentence_embeddings.reshape(1, -1)
    sentence_embeddings = normalize(sentence_embeddings)
    return sentence_embeddings[0]


def compute_doc_embeddings_hf(df: pd.DataFrame, model) -> dict[tuple[str, str], np.ndarray]:
    return {
        idx: get_hf_embeddings(r.content, model) for idx, r in df.iterrows()
    }

In [ ]:
cook_embeddings_hf = compute_doc_embeddings_hf(df, sentence_tf_model)

In [ ]:
df_embedding = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in cook_embeddings_hf.items() ]))
df_embedding.to_csv('embedding.csv')

In [ ]:
def vector_similarity(x: list[float], y: list[float]) -> float:
    """
    Returns the similarity between two vectors.

    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))


def order_document_sections_by_query_similarity(query: str, contexts: dict[(str, str), np.array]) -> list[
    (float, (str, str))]:
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections.

    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_hf_embeddings(query, sentence_tf_model)

    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ], reverse=True)

    return document_similarities

In [ ]:
order_document_sections_by_query_similarity(" What is Abies spectabilis ?", cook_embeddings_hf)[:5]

In [ ]:
MAX_SECTION_LEN = 500
SEPARATOR = "\n* "
ENCODING = "gpt2"  # encoding for text-davinci-003

encoding = tiktoken.get_encoding(ENCODING)
separator_len = len(encoding.encode(SEPARATOR))

f"Context separator contains {separator_len} tokens"

In [ ]:
def construct_prompt(question: str, context_embeddings: dict, df: pd.DataFrame) -> str:
    """
    Fetch relevant
    """
    most_relevant_document_sections = order_document_sections_by_query_similarity(question, context_embeddings)

    chosen_sections = []
    chosen_sections_len = 0
    chosen_sections_indexes = []

    for _, section_index in most_relevant_document_sections:
        # Add contexts until we run out of space.
        document_section = df.loc[section_index]
        document_tokens = len(encoding.encode(document_section.content))
        chosen_sections_len += document_tokens + separator_len
        if chosen_sections_len > MAX_SECTION_LEN:
            break

        chosen_sections.append(SEPARATOR + document_section.content.replace("\n", " "))
        chosen_sections_indexes.append(str(section_index))

    # Useful diagnostic information
    print(f"Selected {len(chosen_sections)} document sections:")
    print("\n".join(chosen_sections_indexes))

    header = """Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."\n\nContext:\n"""

    return header + "".join(chosen_sections) + "\n\n Q: " + question + "\n A:"

In [ ]:
prompt = construct_prompt(
    "What is Abies spectabilis?",
    cook_embeddings_hf,
    df
)

print("===\n", prompt)

In [ ]:
COMPLETIONS_API_PARAMS = {
    # We use temperature of 0.0 because it gives the most predictable, factual answer.
    "temperature": 0.0,
    "max_tokens": 1000,
    "model": COMPLETIONS_MODEL,
}

In [ ]:
def answer_query_with_context(
        query: str,
        df: pd.DataFrame,
        document_embeddings: dict[(str, str), np.array],
        show_prompt: bool = False
) -> str:
    prompt = construct_prompt(
        query,
        document_embeddings,
        df
    )

    if show_prompt:
        print(prompt)

    response = openai.Completion.create(
        prompt=prompt,
        **COMPLETIONS_API_PARAMS
    )

    return response["choices"][0]["text"].strip(" \n")

In [ ]:
answer_query_with_context("What is Abies spectabilis ? Describe it as much as possible",
                          df, cook_embeddings_hf)

Hugging face

In [ ]:
!pip install transformers
!pip install torch

In [ ]:
from transformers import pipeline

qa_model = pipeline("question-answering", model="bert-large-uncased-whole-word-masking-finetuned-squad", tokenizer="bert-large-uncased")


In [ ]:
def answer_query_with_context(query: str, context: str):
    result = qa_model(question=query, context=context)
    return result["answer"]

# Example query
query = "What is Abies spectabilis?"

# Iterate over your paragraphs and find the best answer
best_answer = ""
best_score = 0

for paragraph in paragraphs:
    result = answer_query_with_context(query, paragraph)
    if "answer" in result and "score" in result:
        if result["score"] > best_score:
            best_answer = result["answer"]
            best_score = result["score"]

print("Best Answer:", best_answer)
